#### <font color="#03997A"> Diplomado en Ciencia de datos UNAM </font>

##### <font color="#03997A"> Dr: Alejandro Pimentel  </font>

##### <font color="#03997A"> Modulo 8 Introducción al Deep Learning </font>


##### <font color="#03997A"> Tema 6
Encoder-Decoer & ATENCIÓN Alumno: Ibarra Ramírez Sergio </font>

In [15]:
import tensorflow as tf

In [17]:
num_palabras = 5000
emb_dim = 200
h_dim = 600 # 

In [18]:
# Codificador
entradas_codificador = tf.keras.layers.Input(shape=(None,))
embeddings = tf.keras.layers.Embedding(num_palabras,emb_dim)
palabras_vectorizadas = embeddings(entradas_codificador)

codificador_gru = tf.keras.layers.GRU(h_dim,recurrent_dropout=0.5,return_state=True,return_sequences=True)
salidas_cod, estado_cod = codificador_gru(palabras_vectorizadas)

# Decodificador

decodificador_gru = tf.keras.layers.GRU(h_dim,recurrent_dropout=0.5, return_sequences=True)

salidas_decod = decodificador_gru(salidas_cod,initial_state=estado_cod)

atencion = tf.keras.layers.Attention()
contextos = atencion([salidas_decod,salidas_cod])

resultado_atencion = tf.concat([contextos, salidas_decod], axis=-1)

decodificador_densa = tf.keras.layers.Dense(num_palabras, activation='softmax')

salidas_decod = decodificador_densa(resultado_atencion)

modelo_cod_decod = tf.keras.models.Model(entradas_codificador,salidas_decod)


## Traducción 

In [2]:
archivo_eng_train = "train.en"
archivo_fr_train = "train.fr"

archivo_eng_test = "val.en"
archivo_fr_test = "val.en"

with open(archivo_eng_train) as archivo:
    contenido_eng = archivo.read().lower()
    textos_eng_train = contenido_eng.split('\n')

with open(archivo_fr_train) as archivo:
    contenido_fr = archivo.read().lower()
    textos_fr_train = contenido_fr.split('\n')

with open(archivo_eng_test) as archivo:
    contenido_eng = archivo.read().lower()
    textos_eng_test = contenido_eng.split('\n')

with open(archivo_fr_test) as archivo:
    contenido_fr = archivo.read().lower()
    textos_fr_test = contenido_fr.split('\n')

In [3]:
import nltk

tokenizador = nltk.tokenize.RegexpTokenizer(r'\w+')

textos_eng_train = [tokenizador.tokenize(texto) for texto in textos_eng_train]
textos_fr_train = [tokenizador.tokenize(texto) for texto in textos_fr_train]

textos_eng_test = [tokenizador.tokenize(texto) for texto in textos_eng_test]
textos_fr_test = [tokenizador.tokenize(texto) for texto in textos_fr_test]
textos_fr_test

[['a', 'group', 'of', 'men', 'are', 'loading', 'cotton', 'onto', 'a', 'truck'],
 ['a', 'man', 'sleeping', 'in', 'a', 'green', 'room', 'on', 'a', 'couch'],
 ['a',
  'boy',
  'wearing',
  'headphones',
  'sits',
  'on',
  'a',
  'woman',
  's',
  'shoulders'],
 ['two',
  'men',
  'setting',
  'up',
  'a',
  'blue',
  'ice',
  'fishing',
  'hut',
  'on',
  'an',
  'iced',
  'over',
  'lake'],
 ['a',
  'balding',
  'man',
  'wearing',
  'a',
  'red',
  'life',
  'jacket',
  'is',
  'sitting',
  'in',
  'a',
  'small',
  'boat'],
 ['a',
  'lady',
  'in',
  'a',
  'red',
  'coat',
  'holding',
  'a',
  'bluish',
  'hand',
  'bag',
  'likely',
  'of',
  'asian',
  'descent',
  'jumping',
  'off',
  'the',
  'ground',
  'for',
  'a',
  'snapshot'],
 ['a', 'brown', 'dog', 'is', 'running', 'after', 'the', 'black', 'dog'],
 ['a',
  'young',
  'boy',
  'wearing',
  'a',
  'giants',
  'jersey',
  'swings',
  'a',
  'baseball',
  'bat',
  'at',
  'an',
  'incoming',
  'pitch'],
 ['a',
  'man',
  'in

In [4]:
max_len = max(len(tokens) for tokens in textos_fr_train)
max_len

47

In [5]:
max_oracion = 10

In [19]:
# Quito oraciones cortas para minimizar la predicción de oraciones vacías
indices_train = [i for i,tokens in enumerate(textos_eng_train) if len(tokens) >= max_oracion]

textos_eng_train = [textos_eng_train[i][:max_oracion] for i in indices_train]
textos_fr_train = [textos_fr_train[i][:max_oracion] for i in indices_train]

indices_test = [i for i,tokens in enumerate(textos_eng_test) if len(tokens) >= max_oracion]

textos_eng_test = [textos_eng_test[i][:max_oracion] for i in indices_test]
textos_fr_test = [textos_fr_test[i][:max_oracion] for i in indices_test]

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizador_eng = Tokenizer(num_words=num_palabras,oov_token="UNK")
tokenizador_fr = Tokenizer(num_words=num_palabras,oov_token="UNK")

tokenizador_eng.fit_on_texts(textos_eng_train)
tokenizador_fr.fit_on_texts(textos_fr_train)

X_eng = tokenizador_eng.texts_to_sequences(textos_eng_train)
X_fr = tokenizador_fr.texts_to_sequences(textos_fr_train)

from tensorflow.keras.preprocessing.sequence import pad_sequences

X_eng = pad_sequences(X_eng, maxlen=max_oracion, padding='post')
X_fr = pad_sequences(X_fr, maxlen=max_oracion, padding='post')

In [21]:
X_fr

array([[ 108,   28,    6, ..., 3915,  707,    4],
       [   2,    5,   11, ...,   56,    9,    3],
       [   2,    5,    6, ...,   38,   55,    2],
       ...,
       [  15,  277,  311, ...,   18,  260,  245],
       [   2,  162,    5, ...,  660, 1336,   12],
       [   2,    5,    6, ..., 2061,   29,  463]])

In [22]:
modelo_cod_decod.compile(optimizer="adam",
               loss="sparse_categorical_crossentropy",
               metrics=["accuracy"])

modelo_cod_decod.fit(X_fr,X_eng,batch_size=100,epochs=10)

Epoch 1/10
208/208 [==============================] - 640s 3s/step - loss: 5.0976 - accuracy: 0.2066
Epoch 2/10
208/208 [==============================] - 671s 3s/step - loss: 3.9456 - accuracy: 0.3484
Epoch 3/10
208/208 [==============================] - 703s 3s/step - loss: 3.4187 - accuracy: 0.3953
Epoch 4/10
208/208 [==============================] - 765s 4s/step - loss: 3.0027 - accuracy: 0.4314
Epoch 5/10
208/208 [==============================] - 666s 3s/step - loss: 2.6747 - accuracy: 0.4569
Epoch 6/10
208/208 [==============================] - 698s 3s/step - loss: 2.4011 - accuracy: 0.4820
Epoch 7/10
208/208 [==============================] - 699s 3s/step - loss: 2.1777 - accuracy: 0.5082
Epoch 8/10
208/208 [==============================] - 643s 3s/step - loss: 1.9964 - accuracy: 0.5304
Epoch 9/10
208/208 [==============================] - 710s 3s/step - loss: 1.8490 - accuracy: 0.5527
Epoch 10/10
208/208 [==============================] - 698s 3s/step - loss: 1.7323 - accura

Aplicamos el modelo de tención para traducir un poco de palabras de Frances a Inglés  (dice el profe que esto es hacer trampa pues lo estamos probando con datos de entrnamiento)

In [23]:
Y_pred = modelo_cod_decod.predict(X_fr) # Esto es trampa
Y_pred

650/650 [==============================] - 408s 578ms/step


MemoryError: Unable to allocate 3.87 GiB for an array with shape (20785, 10, 5000) and data type float32

In [ ]:
Y_pred = tf.math.argmax(Y_pred,axis=-1)
Y_pred

In [ ]:
salidas = tokenizador_eng.sequences_to_texts(Y_pred.numpy().tolist())
salidas